### Allows permisions to be set on a file or directory

This is a simple implementation of the `chmod` command in Unix. It allows the user to set the permissions of a file or directory using the octal notation.

```bash
sudo chmod -R 777 /path/to/file
```

In [1]:
import sys
import gc # garbage collector 

import pandas as pd 
from sklearn.model_selection import StratifiedKFold
"""
What is the difference between KFold, StratifiedKFold and StratifiedGroupKFold?
https://stackoverflow.com/questions/45969390/difference-between-kfold-stratifiedkfold-and-stratifiedgroupkfold-in-sklearn

StratifiedKFold is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set.

StratifiedGroupKFold is a variation of k-fold which returns stratified folds, but in addition, the folds are approximately balanced between the different groups.
"""
import numpy as np 
from sklearn.metrics import roc_auc_score 
from sklearn.feature_extraction.text import TfidfVectorizer 

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm 
from transformers import PreTra